### TextBlob (Spacy)

In [3]:
'''INTRO!!!'''

# Catarse bien de cómo funciona.
# Ojo instalaciones (lo he metido en todos laos).

'INTRO!!!'

In [1]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.blob.polarity                            # Polarity: -0.125
doc._.blob.subjectivity                        # Subjectivity: 0.9
doc._.blob.sentiment_assessments.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]
doc._.blob.ngrams()   

[WordList(['I', 'had', 'a']),
 WordList(['had', 'a', 'really']),
 WordList(['a', 'really', 'horrible']),
 WordList(['really', 'horrible', 'day']),
 WordList(['horrible', 'day', 'It']),
 WordList(['day', 'It', 'was']),
 WordList(['It', 'was', 'the']),
 WordList(['was', 'the', 'worst']),
 WordList(['the', 'worst', 'day']),
 WordList(['worst', 'day', 'ever']),
 WordList(['day', 'ever', 'But']),
 WordList(['ever', 'But', 'every']),
 WordList(['But', 'every', 'now']),
 WordList(['every', 'now', 'and']),
 WordList(['now', 'and', 'then']),
 WordList(['and', 'then', 'I']),
 WordList(['then', 'I', 'have']),
 WordList(['I', 'have', 'a']),
 WordList(['have', 'a', 'really']),
 WordList(['a', 'really', 'good']),
 WordList(['really', 'good', 'day']),
 WordList(['good', 'day', 'that']),
 WordList(['day', 'that', 'makes']),
 WordList(['that', 'makes', 'me']),
 WordList(['makes', 'me', 'happy'])]

In [2]:
from textblob import TextBlob

text = "I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy."
blob = TextBlob(text)

print(blob.sentiment_assessments.polarity)
# -0.125

print(blob.sentiment_assessments.subjectivity)
# 0.9

print(blob.sentiment_assessments.assessments)
# [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]


-0.125
0.9
[(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]


### VADER.

VADER (Valence Aware Dictionary and sEntiment Reasoner) es una librería utilizada para el análisis del sentimiento que se enfoca en los textos de social media. Así, pone énfases en las rules que captan la esencia del texto que normalmente se ve en las redes sociales. Realmente está pensado para actuar sobre texto que no han sido limpiados previamente (con emojis, signos de exclamación, etc.). Nosotros lo probaremos tanto sobre los datos ya limpios, como sobre los datos originales. Lo ideal sería evaluar la precisión de cada uno de ellos (junto con el análisis desarrollado mediante otros modelos), para identificar el que ofrece un mayor rendimiento.

https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [ ]:
from src.nlp_functions import *

In [ ]:
import pandas as pd

brands_tweets = pd.read_pickle('data/brands_tokens.pkl')

In [ ]:
brands_tweets.head()

In [ ]:
brands_tweets['vader_scores'] = brands_tweets.token.apply(vader_scoring)

In [ ]:
brands_tweets.sample(10)